# Setting up

In [56]:
%load_ext autoreload
%autoreload 2
%pip install nltk pandas
import nltk
nltk.download('punkt')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
You should consider upgrading via the '/home/gitpod/.pyenv/versions/3.9.7/envs/search_with_ml_week3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt to /home/gitpod/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [258]:
import pandas as pd
import json
import contextlib
from functions import display_rows

ENABLE_CLASSIFIER_TRAINING=False
ENABLE_SYNONYM_TRAINING=False


%env WEEK3 /workspace/search_with_machine_learning_course/week3
WEEK3='/workspace/search_with_machine_learning_course/week3'

%env MAX_CATEGORY_DEPTH=4
MAX_CATEGORY_DEPTH=4

PRUNE_PRODUCT_IF_LESS_THAN = 5


env: WEEK3=/workspace/search_with_machine_learning_course/week3
env: MAX_CATEGORY_DEPTH=4


# Training Classifier

In [259]:
# Dump categories, 10 level deep, dump everything
!python dump_category_depth_maps.py --max-depth 10
!head -n 3 $WEEK3/data/category_depth_maps.csv

                                           name  depth
id                                                    
cat00000                               Best Buy      1
abcat0010000                        Gift Center      2
abcat0011000                                Her      3
abcat0011001                      Leisure Gifts      4
abcat0011002                 Kitchen Essentials      4
...                                         ...    ...
pcmcat97200050007      Sony Digital SLR Cameras      5
pcmcat97200050013  Sony Digital SLR Accessories      5
pcmcat97200050015              Sony DSLR Camera      4
pcmcat99000050002              Dynex Networking      5
pcmcat99300050000    Sony Digital SLR Camcorder      4

[4017 rows x 2 columns]
Number of categories: 4017
Writting data to /workspace/search_with_machine_learning_course/week3/data/category_depth_maps.csv...
id,name,depth
cat00000,Best Buy,1
abcat0010000,Gift Center,2


In [260]:
category_depth_maps_df = pd.read_csv(f'{WEEK3}/data/category_depth_maps.csv').set_index('id')
category_depth_maps_at_depth_df = category_depth_maps_df[category_depth_maps_df['depth'] <= MAX_CATEGORY_DEPTH]

display_rows(category_depth_maps_at_depth_df, 4)


,name,depth
id,,
cat00000,Best Buy,1
abcat0010000,Gift Center,2
...,...,...
pcmcat97200050015,Sony DSLR Camera,4
pcmcat99300050000,Sony Digital SLR Camcorder,4


In [261]:
#!python createContentTrainingData.py --output $WEEK3/data/output.fasttext

In [262]:
!tail -n 3 $WEEK3/data/output.fasttext
!tail -n 3 $WEEK3/data/output.fasttext.csv

__label__pcmcat202500050007 eric clapton clapton acoust dvd
__label__pcmcat202500050007 greg koch guitar gristl dvd
__label__pcmcat202500050007 pro tool le 7 0 beginn level dvd
115355,pcmcat202500050007,Eric Clapton: Clapton Acoustic - DVD,eric clapton clapton acoust dvd
115356,pcmcat202500050007,Greg Koch Guitar Gristle - DVD,greg koch guitar gristl dvd
115357,pcmcat202500050007,Pro Tools LE 7.0: Beginner Level - DVD,pro tool le 7 0 beginn level dvd


In [263]:
# load all fastext labels
fasttext_df = pd.read_csv(f'{WEEK3}/data/output.fasttext.csv').set_index('label')

# shuffle data
fasttext_df = fasttext_df.sample(frac=1)

# group data by labels
fasttext_df_group_by_label = fasttext_df.groupby('label')
print(f'fasttext_df_group_by_label:')
display_rows(fasttext_df_group_by_label.agg(['count']), 5)


fasttext_df_group_by_label:


,Unnamed: 0,name,name_stemmed
,count,count,count
label,,,
abcat0011000,12,12,12
abcat0011001,8,8,8
...,...,...,...
pcmcat97200050013,3,3,3
pcmcat99000050002,6,6,6


In [264]:
# prune the labels that has less than N products
fasttext_df_labels_with_more_than_n_products = fasttext_df_group_by_label.filter(lambda x: x.shape[0] > PRUNE_PRODUCT_IF_LESS_THAN)
print(f'fasttext_df_labels_with_more_than_{PRUNE_PRODUCT_IF_LESS_THAN}_products:')
display_rows(fasttext_df_labels_with_more_than_n_products, 5)

with open(f'{WEEK3}/data/output-prune-less-than-{PRUNE_PRODUCT_IF_LESS_THAN}-products.fasttext', 'w') as output:
    for index, row in fasttext_df_labels_with_more_than_n_products.iterrows():        
        output.write("__label__%s %s\n" % (index, row['name_stemmed']))


fasttext_df_labels_with_more_than_5_products:


,Unnamed: 0,name,name_stemmed
label,,,
abcat0511004,90924,Brother - Wireless Multifunction Printer/ Copi...,brother wireless multifunct printer copier sca...
abcat0704003,73402,Prince of Persia: The Sands of Time - PlayStat...,princ of persia the sand of time playstat 2
...,...,...,...
abcat0706002,42982,Death Jr.: Root of Evil - Nintendo Wii,death jr root of evil nintendo wii
abcat0916009,110285,BISSELL - 22-oz. Tough Stain Precleaner,bissel 22 oz tough stain preclean


In [265]:

# keep only the labels up to depth Nth catetory depth. For example, N=2, Best Buy -> Category level2
fasttext_df_at_depth = fasttext_df[
    fasttext_df.index.isin(category_depth_maps_at_depth_df.index)
]
fasttext_df_at_depth_first_cell_id = fasttext_df_at_depth.index.values[0]
print('fasttext_df_at_depth:')
display_rows(fasttext_df_at_depth, 5)
print(f'The depth of {fasttext_df_at_depth_first_cell_id} in category_depth_maps_at_depth_df depth (for UAT):')
display(
category_depth_maps_at_depth_df[category_depth_maps_at_depth_df.index == fasttext_df_at_depth_first_cell_id]
)

with open(f'{WEEK3}/data/output-prune-category-at-depth-{MAX_CATEGORY_DEPTH}.fasttext', 'w') as output:
    for index, row in fasttext_df_at_depth.iterrows():        
        output.write("__label__%s %s\n" % (index, row['name_stemmed']))

print('')

fasttext_df_at_depth:


,Unnamed: 0,name,name_stemmed
label,,,
abcat0511004,90924,Brother - Wireless Multifunction Printer/ Copi...,brother wireless multifunct printer copier sca...
abcat0201010,107036,GPX - 4GB* Video MP3 Player and Image Viewer -...,gpx 4gb video mp3 player and imag viewer black
...,...,...,...
pcmcat144700050004,95680,Klipsch - Image S3 Earbud Headphones - Rebel Red,klipsch imag s3 earbud headphon rebel red
abcat0706002,42982,Death Jr.: Root of Evil - Nintendo Wii,death jr root of evil nintendo wii


The depth of abcat0511004 in category_depth_maps_at_depth_df depth (for UAT):


,name,depth
id,,
abcat0511004,All-In-One Inkjet Printers,4


In [266]:
# keep only the labels up to depth Nth catetory depth and prune labels with less than N products
fasttext_df_more_than_n_products_and_at_depth = fasttext_df_labels_with_more_than_n_products[
    fasttext_df_labels_with_more_than_n_products.index.isin(category_depth_maps_at_depth_df.index)
]

print('')

with open(f'{WEEK3}/data/output-prune-less-than-{PRUNE_PRODUCT_IF_LESS_THAN}-prune-category-at-depth-{MAX_CATEGORY_DEPTH}.fasttext', 'w') as output:
    for index, row in fasttext_df_at_depth.iterrows():        
        output.write("__label__%s %s\n" % (index, row['name_stemmed']))


In [271]:

%env DATA_SIZE=25000
%env CLASSIFICATION_TRAINING_FILE=$WEEK3/data/output-prune-less-than-5-prune-category-at-depth-4.fasttext
%env FASTTEXT_TRAINING_FILE=/workspace/search_with_machine_learning_course/week3/data/data.train
%env FASTTEXT_TEST_FILE=/workspace/search_with_machine_learning_course/week3/data/data.test


!mkdir -p /workspace/search_with_machine_learning_course/week3/data
print('Generating data.train...')
!head -n $DATA_SIZE $CLASSIFICATION_TRAINING_FILE > $FASTTEXT_TRAINING_FILE
print('Done')
print('Generating data.test...')
!tail -n $DATA_SIZE $CLASSIFICATION_TRAINING_FILE > $FASTTEXT_TEST_FILE
print('Done')

print('\n')
!echo Fist 5 lines in $FASTTEXT_TRAINING_FILE
!head -n 5 $FASTTEXT_TRAINING_FILE
print('\n')
!echo Fist 5 lines in $FASTTEXT_TEST_FILE
!head -n 5 $FASTTEXT_TEST_FILE



env: DATA_SIZE=25000
env: CLASSIFICATION_TRAINING_FILE=/workspace/search_with_machine_learning_course/week3/data/output-prune-less-than-5-prune-category-at-depth-4.fasttext
env: FASTTEXT_TRAINING_FILE=/workspace/search_with_machine_learning_course/week3/data/data.train
env: FASTTEXT_TEST_FILE=/workspace/search_with_machine_learning_course/week3/data/data.test
Generating data.train...
Done
Generating data.test...
Done


Fist 5 lines in /workspace/search_with_machine_learning_course/week3/data/data.train
__label__abcat0511004 brother wireless multifunct printer copier scanner fax photocaptur center
__label__abcat0201010 gpx 4gb video mp3 player and imag viewer black
__label__abcat0808006 hewlett packard 50 pack 8 5 x 11 matt photo paper
__label__abcat0910004 whirlpool 4 cycl extra larg capac electr dryer bisqu
__label__abcat0504010 pni attaché 4gb usb 2 0 flash drive


Fist 5 lines in /workspace/search_with_machine_learning_course/week3/data/data.test
__label__pcmcat174700050005 the sim 

In [296]:
!~/fastText-0.9.2/fasttext supervised \
    -input $FASTTEXT_TRAINING_FILE \
    -output /workspace/search_with_machine_learning_course/week3/data/model \
    -epoch 25 \
    -lr 1 \
    -loss hs \
    -wordNgrams 2


!~/fastText-0.9.2/fasttext test \
    /workspace/search_with_machine_learning_course/week3/data/model.bin \
    $FASTTEXT_TEST_FILE


Read 0M words
Number of words:  10617
Number of labels: 637
Progress: 100.0% words/sec/thread:   21662 lr:  0.000000 avg.loss:  0.267559 ETA:   0h 0m 0s14s% words/sec/thread:   21603 lr:  0.114491 avg.loss:  0.338832 ETA:   0h 0m 6s
N	24902
P@1	0.842
R@1	0.842


In [297]:
!python f_classify.py -n 5 -i "apple"

input_stemmed: appl
Mobile Phone Add-Ons
iPhone Appcessories&#x2122; Products
Shavers &amp; Trimmers
Video Graphics Cards
PlayStation 3


# Training Synonym

In [304]:
!~/fastText-0.9.2/fasttext skipgram \
    -input /workspace/datasets/fasttext/titles.txt \
    -output /workspace/search_with_machine_learning_course/week3/data/title_model_epoch_25

Read 0M words
Number of words:  2318
Number of labels: 0
Progress:   0.0% words/sec/thread:     429 lr:  0.999814 avg.loss: 81.661057 ETA:   0h 9m 0sterminate called after throwing an instance of 'fasttext::DenseMatrix::EncounteredNaNError'
  what():  Encountered NaN.


In [315]:
import fasttext
import functions
from hand_picked_tokens import get_tokens

models = [
    'title_model.bin'
]

output = ''
for input in get_tokens():
    for model_name in models:
        input_stemmed = functions.transform_name(input)
        model = fasttext.load_model(f'/workspace/search_with_machine_learning_course/week3/data/{model_name}')    
        predictions = model.get_nearest_neighbors(input_stemmed, k=10)

        output += f'Synonyms of [{input}]:\n'
        
        for (score, r) in predictions:    
            output += f'    {r}    ({score})\n'
        output += '----------------------------\n\n'


print(output)

Synonyms of [Sony]:
    fujitsu    (0.9550379514694214)
    hdr    (0.9518088102340698)
    memorex    (0.9482449293136597)
    sdr    (0.9457224607467651)
    cyberpowerpc    (0.9444787502288818)
    sodimm    (0.9435911178588867)
    sd    (0.9379982352256775)
    vixia    (0.9368028044700623)
    everio    (0.9351425766944885)
    16gb    (0.9324381351470947)
----------------------------

Synonyms of [apple]:
    generat    (0.9855630397796631)
    ip    (0.983881950378418)
    ipad    (0.9790254235267639)
    4th    (0.9736116528511047)
    iphon    (0.9729768633842468)
    3rd    (0.9725083708763123)
    ipod    (0.9614014625549316)
    nano    (0.9594425559043884)
    touch    (0.9512611031532288)
    dock    (0.9455647468566895)
----------------------------

Synonyms of [iphone]:
    appl    (0.972976565361023)
    generat    (0.9585882425308228)
    4th    (0.956022322177887)
    ip    (0.9500055909156799)
    3rd    (0.9484606981277466)
    ipad    (0.9468241333961487)
    4s 

In [5]:
token='iphone'
print(f'Synonyms for {token}:')
predictions = f_nn.predict(token, 10)
for (score, r) in predictions:    
    print(f'    {r}    ({score})')
print('----------------------------')
print('')

Synonyms for iphone:
    appl    (0.9709902405738831)
    generat    (0.9644677042961121)
    4th    (0.9544933438301086)
    3rd    (0.9525007009506226)
    ip    (0.9490216374397278)
    ipad    (0.9462820887565613)
    skin    (0.9401151537895203)
    jack    (0.9356447458267212)
    headphon    (0.9349609613418579)
    charg    (0.9332101941108704)
----------------------------



In [1]:
import hand_picked_tokens
import f_nn

for token in hand_picked_tokens.get_tokens():
    print(f'Synonyms for {token}:')
    predictions = f_nn.predict(token, 10)
    for (score, r) in predictions:    
        print(f'    {r}    ({score})')
    print('----------------------------')
    print('')

Synonyms for Sony:
    memorex    (0.9596976041793823)
    hdr    (0.9518812298774719)
    fujitsu    (0.9491270780563354)
    sodimm    (0.9482216835021973)
    sdr    (0.9454458951950073)
    cyberpowerpc    (0.9432742595672607)
    sd    (0.9415234327316284)
    vixia    (0.9386047720909119)
    everio    (0.9351485967636108)
    pni    (0.9347209334373474)
----------------------------

Synonyms for apple:
    ip    (0.9880794882774353)
    generat    (0.9848724603652954)
    ipad    (0.9772943258285522)
    4th    (0.9728609919548035)
    iphon    (0.9709898829460144)
    3rd    (0.9698827266693115)
    nano    (0.9561010599136353)
    touch    (0.953309953212738)
    ipod    (0.9510629177093506)
    dock    (0.9477444291114807)
----------------------------

Synonyms for iphone:
    appl    (0.9709902405738831)
    generat    (0.9644677042961121)
    4th    (0.9544933438301086)
    3rd    (0.9525007009506226)
    ip    (0.9490216374397278)
    ipad    (0.9462820887565613)
    skin 

# Review rating to language correlation



In [53]:
%env REVIEW_DATA_SIZE=50000
%env RAW_FASTTEXT_REVIEW_LABEL_FILE=/workspace/search_with_machine_learning_course/week3/data/output-reviews.fasttext
%env SHUFFLED_FASTTEXT_REVIEW_LABEL_FILE=/workspace/search_with_machine_learning_course/week3/data/output-reviews-shuffled.fasttext
%env FASTTEXT_REVIEW_TRAINING_FILE=/workspace/search_with_machine_learning_course/week3/data/data-review.train
%env FASTTEXT_REVIEW_TEST_FILE=/workspace/search_with_machine_learning_course/week3/data/data-review.test

env: REVIEW_DATA_SIZE=50000
env: RAW_FASTTEXT_REVIEW_LABEL_FILE=/workspace/search_with_machine_learning_course/week3/data/output-reviews.fasttext
env: SHUFFLED_FASTTEXT_REVIEW_LABEL_FILE=/workspace/search_with_machine_learning_course/week3/data/output-reviews-shuffled.fasttext
env: FASTTEXT_REVIEW_TRAINING_FILE=/workspace/search_with_machine_learning_course/week3/data/data-review.train
env: FASTTEXT_REVIEW_TEST_FILE=/workspace/search_with_machine_learning_course/week3/data/data-review.test


In [50]:
!python createReviewLabels.py --use-3-tie --output $RAW_FASTTEXT_REVIEW_LABEL_FILE

use_3_tier: True
Writing results to /workspace/search_with_machine_learning_course/week3/data/output-reviews.fasttext


In [54]:
!wc -l $RAW_FASTTEXT_REVIEW_LABEL_FILE

!echo ""
!echo "Shuffle $RAW_FASTTEXT_REVIEW_LABEL_FILE into $SHUFFLED_FASTTEXT_REVIEW_LABEL_FILE"
!shuf $RAW_FASTTEXT_REVIEW_LABEL_FILE -o $SHUFFLED_FASTTEXT_REVIEW_LABEL_FILE

#!echo ""
#!echo "First first 3 rows of $RAW_FASTTEXT_REVIEW_LABEL_FILE"
#!head -n 3 $RAW_FASTTEXT_REVIEW_LABEL_FILE

#!echo ""
#!echo "First first 3 rows of $SHUFFLED_FASTTEXT_REVIEW_LABEL_FILE"
#!head -n 3 $SHUFFLED_FASTTEXT_REVIEW_LABEL_FILE



221771 /workspace/search_with_machine_learning_course/week3/data/output-reviews.fasttext

Shuffle /workspace/search_with_machine_learning_course/week3/data/output-reviews.fasttext into /workspace/search_with_machine_learning_course/week3/data/output-reviews-shuffled.fasttext


In [52]:

!mkdir -p /workspace/search_with_machine_learning_course/week3/data

!echo "Generating $FASTTEXT_REVIEW_TRAINING_FILE..."
!head -n $REVIEW_DATA_SIZE $SHUFFLED_FASTTEXT_REVIEW_LABEL_FILE > $FASTTEXT_REVIEW_TRAINING_FILE
print('Done')

!echo "Generating $FASTTEXT_REVIEW_TEST_FILE..."
!tail -n $REVIEW_DATA_SIZE $SHUFFLED_FASTTEXT_REVIEW_LABEL_FILE > $FASTTEXT_REVIEW_TEST_FILE
print('Done')


#print('\n')
#!echo Fist 5 lines in $FASTTEXT_REVIEW_TRAINING_FILE
#!head -n 5 $FASTTEXT_REVIEW_TRAINING_FILE
#print('\n')
#!echo Fist 5 lines in $FASTTEXT_REVIEW_TEST_FILE
#!head -n 5 $FASTTEXT_REVIEW_TEST_FILE


Generating /workspace/search_with_machine_learning_course/week3/data/data-review.train...
Done
Generating /workspace/search_with_machine_learning_course/week3/data/data-review.test...
Done


In [55]:
if True:
    !~/fastText-0.9.2/fasttext supervised \
        -input $FASTTEXT_REVIEW_TRAINING_FILE \
        -output $WEEK3/data/review-model \
        -autotune-validation $FASTTEXT_REVIEW_TEST_FILE \
        -autotune-duration 1200


    !~/fastText-0.9.2/fasttext test \
        $WEEK3/data/review-model.bin \
        $FASTTEXT_REVIEW_TEST_FILE

Progress: 100.0% Trials:    8 Best score:  0.895900 ETA:   0h 0m 0s ETA:   0h15m57s% Trials:    5 Best score:  0.887780 ETA:   0h10m 8ss% Trials:    8 Best score:  0.895900 ETA:   0h 0m47s
Training again with best arguments
Read 4M words
Number of words:  128676
Number of labels: 3
Progress: 100.0% words/sec/thread:   39975 lr:  0.000000 avg.loss:  0.042728 ETA:   0h 0m 0s44sss  0h 4m46s 4m41s  0h 4m36s 14.6% words/sec/thread:   40152 lr:  0.169445 avg.loss:  0.236330 ETA:   0h 4m30ssh 3m39sm36s34ss 42.6% words/sec/thread:   40013 lr:  0.113903 avg.loss:  0.094284 ETA:   0h 3m 2ssh 2m48s15s  0h 1m58ss  39975 lr:  0.032972 avg.loss:  0.050787 ETA:   0h 0m52s51ss
N	50000
P@1	0.896
R@1	0.896


In [9]:
if True:
    !~/fastText-0.9.2/fasttext supervised \
        -input $FASTTEXT_REVIEW_TRAINING_FILE \
        -output $WEEK3/data/review-model \
        -epoch 25 \
        -lr 1.0 \
        -wordNgrams 2


    !~/fastText-0.9.2/fasttext test \
        $WEEK3/data/review-model.bin \
        $FASTTEXT_REVIEW_TEST_FILE

Read 0M words
Number of words:  5139
Number of labels: 3
Progress: 100.0% words/sec/thread:   60481 lr:  0.000000 avg.loss:  0.073778 ETA:   0h 0m 0s
N	1000
P@1	0.863
R@1	0.863
